# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
orders = pd.read_csv('Orders.csv')
print(orders.head())


   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# Q1: Aggregate amount_spent per customer and label VIP/Preferred customers
# Group by CustomerID and Country and compute total amount_spent
customer_spend = orders.groupby(['CustomerID', 'Country'])['amount_spent'].sum().reset_index()

# Compute quantile thresholds for VIP and Preferred. We'll define VIP as top 10% and Preferred as next 20%
vip_threshold = customer_spend['amount_spent'].quantile(0.90)
pref_threshold = customer_spend['amount_spent'].quantile(0.70)

# Label customers based on spend

def label_customer(amount):
    if amount >= vip_threshold:
        return 'VIP'
    elif amount >= pref_threshold:
        return 'Preferred'
    else:
        return 'Regular'

customer_spend['Customer_Type'] = customer_spend['amount_spent'].apply(label_customer)

# Display the distribution of customer types
print(customer_spend['Customer_Type'].value_counts())

# Show a sample of labeled customers
customer_spend.head()

Customer_Type
Regular      3043
Preferred     869
VIP           435
Name: count, dtype: int64


,CustomerID,Country,amount_spent,Customer_Type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,VIP
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [5]:
# Q2: Country with the most VIP customers
vip_counts = customer_spend[customer_spend['Customer_Type'] == 'VIP'].groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
print(vip_counts.head())

Country
United Kingdom    356
Germany            23
France             18
Spain               6
Switzerland         5
Name: CustomerID, dtype: int64


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [6]:
# Q3: Country with the most VIP and Preferred customers combined
vip_pref_counts = customer_spend[customer_spend['Customer_Type'].isin(['VIP', 'Preferred'])].groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
print(vip_pref_counts.head())

Country
United Kingdom    1135
Germany             42
France              30
Switzerland         11
Belgium             11
Name: CustomerID, dtype: int64
